In [ ]:
import numpy as np

***
## Configuration
***

In [ ]:
# PARAMETERS
# Modify this section to adjust the classifier's behavior.
shots = 8192
dataset_size = 512
training_size = 0.5
n_trash_qubits = 3

batch_size = 25
iters = int((dataset_size * training_size) / batch_size)

In [ ]:
# SIMULATION CONFIG
# Modify this section to change the type of simulation used.
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime.fake_provider import FakeNairobiV2

noise_model = NoiseModel.from_backend(FakeNairobiV2())
backend = AerSimulator(noise_model=noise_model)

***
## Dataset
***

In [5]:
from schmidt_classifier import load_fraud_dataset

features, labels = load_fraud_dataset(dataset_size, 42, False)

feature_dim = len(features[0])
n_qubits = int(np.ceil(np.log2(feature_dim)))

***
## Experiment
***

In [ ]:
from qiskit.circuit.library import StatePreparation

import warnings
warnings.simplefilter('ignore')

from schmidt_classifier import configure_compressor, extract_features, training, testing

def full_experiment(target_class, n_trash_partition, reps=1):

    print('configuring compressor...')

    # 1. CONFIGURE COMPRESSOR
    compressor = configure_compressor(
        features, labels, target_class, n_qubits, n_trash_partition, training_size, backend, verbose=1
    )

    print('extracting features...')

    # 2. CREATE DATASETS FROM EXTRACTED FEATURES (TRASH DATA)
    train_data, train_targets, test_data, test_targets = extract_features(
        features, labels, target_class, compressor, n_qubits, training_size, backend, shots, initializer, opt_params, verbose=0
    )

    print('training and testing...')

    # REPEAT TRAINING AND TESTING PROCESS `reps` TIMES
    metrics = ([],[],[],[],[],[],[],[],[],[],[])

    print(f'\trep:', end='')
    for rep in range(reps):
        print(f' {rep} ', end='')

        # 3. TRAINING
        model = training(train_data, train_targets, iters, batch_size, verbose=0)

        # 4. TESTING
        total_loss, tp, tn, acc, fp, fn, f1, mcc, scores, auc = testing(model, test_data, test_targets, verbose=0)

        # SAVES TESTING RESULTS FOR EACH `rep`
        metrics[0].append(sum(total_loss))
        metrics[1].append(tp)
        metrics[2].append(tn)
        metrics[3].append(acc)
        metrics[4].append(fp)
        metrics[5].append(fn)
        metrics[6].append(f1)
        metrics[7].append(mcc)
        metrics[8].append(scores)
        metrics[9].append(test_targets)
        metrics[10].append(auc)
    
    print()

    return metrics


results = {}
target_class = 0
for max_fidelity_loss in [0.0, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.60, 1.0]:
    print('max_fidelity_loss:', max_fidelity_loss)

    if max_fidelity_loss == 0.0:
        opt_params = None
        initializer = StatePreparation
    else:
        from q_alchemy.qiskit_integration import QAlchemyInitialize, OptParams
        opt_params = OptParams(
            max_fidelity_loss=max_fidelity_loss,
            basis_gates=['u', 'cx'],
            api_key=""
        )
        initializer = QAlchemyInitialize

        # from qclib.state_preparation import BaaLowRankInitialize
        # opt_params = {
        #     'max_fidelity_loss': max_fidelity_loss,
        #     'strategy': 'brute_force',
        #     'use_low_rank': True
        # }
        # initializer = BaaLowRankInitialize

    results[max_fidelity_loss] = full_experiment(target_class, n_trash_partition=n_trash_qubits, reps=50)

max_fidelity_loss: 0.0
configuring compressor...
	trash qubits: 3
	trash_partition: [0, 2, 4]
	compressor depth: 52
	compressor cnots: 20
extracting features...
training and testing...
	rep: 0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49 
max_fidelity_loss: 0.1
configuring compressor...
	trash qubits: 3
	trash_partition: [0, 2, 4]
	compressor depth: 52
	compressor cnots: 20
extracting features...
training and testing...
	rep: 0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49 
max_fidelity_loss: 0.15
configuring compressor...
	trash qubits: 3
	trash_partition: [0, 2, 4]
	compressor depth: 52
	compressor cnots: 20
extracting features...
training and testing...
	rep: 0  1  2  3  4  5  6  7  8  9  10  11  12  13  

In [7]:
from schmidt_classifier import display_results
display_results(results)

| max loss | avg. TP | std. TP | avg. TN | std. TN | avg. FP | std. FP | avg. FN | std. FN | avg. acc | std. acc | avg. F1 | std. F1 | avg. MCC | std. MCC | avg. AUC | std. AUC |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 0.0 | 91.9 | 18.64 | 62.02 | 22.7161 | 65.98 | 22.7161 | 36.1 | 18.64 | 0.6012 | 0.0269 | 0.6362 | 0.0591 | 0.219 | 0.0422 | 0.6738 | 0.0113 |
| 0.1 | 89.8 | 24.9664 | 65.96 | 28.9289 | 62.04 | 28.9289 | 38.2 | 24.9664 | 0.6084 | 0.0341 | 0.6267 | 0.1022 | 0.2411 | 0.0551 | 0.6846 | 0.0226 |
| 0.15 | 97.14 | 18.9304 | 62.16 | 25.9733 | 65.84 | 25.9733 | 30.86 | 18.9304 | 0.6223 | 0.041 | 0.663 | 0.0447 | 0.268 | 0.0642 | 0.6915 | 0.0239 |
| 0.2 | 98.2 | 16.4669 | 62.98 | 22.1029 | 65.02 | 22.1029 | 29.8 | 16.4669 | 0.6296 | 0.0336 | 0.6707 | 0.0395 | 0.2808 | 0.0538 | 0.6998 | 0.0266 |
| 0.25 | 94.44 | 16.6927 | 69.4 | 18.5159 | 58.6 | 18.5159 | 33.56 | 16.6927 | 0.64 | 0.0184 | 0.6673 | 0.0435 | 0.2977 | 0.0281 | 0.711 | 0.0105 |
| 0.3 | 96.52 | 16.4295 | 58.94 | 19.4714 | 69.06 | 19.4714 | 31.48 | 16.4295 | 0.6073 | 0.0245 | 0.6535 | 0.0399 | 0.2342 | 0.0426 | 0.6673 | 0.015 |
| 0.35 | 103.62 | 12.8699 | 54.18 | 19.9967 | 73.82 | 19.9967 | 24.38 | 12.8699 | 0.6164 | 0.0315 | 0.6771 | 0.0202 | 0.2599 | 0.0451 | 0.6886 | 0.0211 |
| 0.4 | 89.96 | 18.3641 | 70.72 | 19.1333 | 57.28 | 19.1333 | 38.04 | 18.3641 | 0.6277 | 0.017 | 0.647 | 0.0544 | 0.2706 | 0.0273 | 0.6858 | 0.0079 |
| 0.45 | 84.0 | 17.3678 | 75.9 | 19.901 | 52.1 | 19.901 | 44.0 | 17.3678 | 0.6246 | 0.0182 | 0.6297 | 0.0511 | 0.2616 | 0.0334 | 0.6907 | 0.0082 |
| 0.5 | 99.4 | 13.8087 | 59.88 | 21.3669 | 68.12 | 21.3669 | 28.6 | 13.8087 | 0.6222 | 0.0369 | 0.671 | 0.0261 | nan | nan | 0.6871 | 0.0146 |
| 0.6 | 104.18 | 9.5931 | 54.42 | 14.2086 | 73.58 | 14.2086 | 23.82 | 9.5931 | 0.6195 | 0.0246 | 0.6807 | 0.018 | 0.2635 | 0.0394 | 0.6784 | 0.0163 |
| 1.0 | 100.26 | 17.6995 | 59.0 | 17.317 | 69.0 | 17.317 | 27.74 | 17.6995 | 0.6221 | 0.0243 | 0.6682 | 0.0628 | 0.269 | 0.0458 | 0.671 | 0.0217 |
